In [1]:
import pandas as pd
import numpy as np
import rpy2

In [ ]:
bodysite = 'Breast'
info = pd.read_csv('../data/TOIL/sample_info.txt', 
                   sep='\t', header=0, index_col=0)

In [3]:
whole = pd.read_csv('../data/TOIL/TcgaTargetGtex_RSEM_Hugo_norm_count.txt', 
                    sep='\t', header=0, index_col=0)
whole = np.power(2, whole) - 1
whole[info[info['body_site'] == bodysite].index].to_csv('../data/TOIL/TOIL_RSEM_Hugo_{0}_count_float.txt'.format(bodysite), sep='\t')

In [2]:
bodysite = 'Breast'
info = pd.read_csv('../data/TOIL/sample_info.txt', 
                   sep='\t', header=0, index_col=0)
expr = pd.read_csv('../data/TOIL/TOIL_RSEM_Hugo_Breast_count_int.txt', 
                   sep='\t', header=0, index_col=0)
expr = expr.dropna()

expressed = expr.apply(lambda x: x > 10).agg(lambda x: np.sum(x) >= 2, axis=1)
expressed_genes = expr[expressed].index.values


# After RUVSeq & edgeR

In [40]:
def deg(df, strict=True):
    """Returns UP DOWN Series"""
    df = df[df['logCPM'] > 3]
    if not strict:
        return df['logFC'].apply(lambda x: int(np.sign(x)))
    
    return df['logFC'].apply(lambda x: int(np.sign(x) * int(np.abs(x) > 1)))

In [41]:
expr_models = ['UD', 'DU', 'UU', 'DD', 'US', 'DS', 'SU', 'SD', 'SS']

In [57]:
tmp_df = pd.DataFrame(0, index=expressed_genes, columns=['N_P', 'P_T', 'N_T'])
for a, b in [('Normal', 'PNT'), ('PNT', 'Tumor'), ('Normal', 'Tumor')]:
    print a, b
    df = pd.read_csv('../Analysis/DEG/{0}.{1}_{2}.DEG_list.txt'.format(bodysite, a, b),
                 sep='\t', header=0, index_col=0)
    
    tmp_df.loc[:, '{0}_{1}'.format(a[0], b[0])] = deg(df, strict=True)
tmp_df = tmp_df.fillna(0)
tmp_df = tmp_df.astype('int64')

Normal PNT
PNT Tumor
Normal Tumor


In [45]:
grid = pd.DataFrame(np.array([[1, -1, 0],
                              [-1, 1, 0],
                              [1, 1, 1],
                              [-1, -1, -1],
                              [1, 0, 1],
                              [-1, 0, -1],
                              [0, -1, -1],
                              [0, 1, 1],
                              [0, 0, 0]]),
                    index=expr_models,
                    columns=['N_P', 'P_T', 'N_T'])

In [58]:
model_ser = pd.Series(index=tmp_df.index)
for i in range(tmp_df.shape[0]):
    for k in range(grid.shape[0]):
        if np.array_equal(tmp_df.iloc[i, :].values, grid.iloc[k, :].values):
            model_ser.iloc[i] = grid.index[k]

In [59]:
model_ser = model_ser.dropna()

In [60]:
model_ser['MUC1']

'SS'

In [61]:
for i in expr_models:
    print i, model_ser[model_ser == i].shape[0]

UD 1
DU 0
UU 1
DD 0
US 14
DS 4
SU 34
SD 398
SS 34704


In [62]:
model_ser.to_csv('../Analysis/DEG/expression_models.2fold.txt', sep='\t', header=False)